# Testing single node datasets

Purpose: until now dataset represented by combination of tree types - now test for single node type

## Importing

In [2]:
import sys
import os

# Construct the path to the parent directory of the parent directory
parent_of_parent_dir = os.path.abspath(os.path.join(os.path.pardir, os.path.pardir))

# Add the parent directory of the parent directory to the system path
sys.path.insert(0, parent_of_parent_dir)

In [3]:
import optuna
import numpy as np
from sklearn.metrics import r2_score
import torch
from torch.utils.data import random_split, DataLoader

from etnn import TreeNode
from etnn.tools.training_tools import ConfigStore
from etnn.data.ferris_wheel import load_pure_ferris_wheel_dataset_single_node
from etnn.tools.training import train_epoch, eval_epoch
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.tools.training_tools import seeding_all
from etnn.routines.run_config import choice_trainloader, choice_loss, choice_optim

## Control constants

In [4]:
dataset_path = "../../datasets/"
test_perc = 0.3
val_perc = 0.21
stability_count = 5
label = "tree-advanced" # although irrelevant

## Defining parameter search

In [5]:
def objective(trial):
    # init default config
    config = ConfigStore(
       in_dim=15,
       hidden_dim=trial.suggest_int("hidden_dim", 16, 256, step=16),
       out_dim=1,
       k=trial.suggest_int("k", 1, 3),
       dataset=-1 if normalized else 0,
       ds_size=10_000,
       num_gondolas=-1,
       num_part_pg=-1,
       loss_name='mse',
       optimizer_name='adam',
       num_max_epochs=30, # real: 100
       learning_rate=trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True),
       batch_size=1024,
       early_stop_tol=5,
       use_equal_batcher=trial.suggest_categorical("batcher", [True, False]),
       seed=420,
       label_type=label,
       final_label_factor=5/1000
    )
    # config = ConfigStore(
    #     in_dim=15,
    #     hidden_dim=16,
    #     out_dim=1,
    #     k=2,
    #     dataset=-1 if normalized else 0,
    #     ds_size=10_000,
    #     num_gondolas=-1,
    #     num_part_pg=-1,
    #     loss_name='mse',
    #     optimizer_name='adam',
    #     num_max_epochs=30, # real: 100
    #     learning_rate=0.002,
    #     batch_size=1024,
    #     early_stop_tol=5,
    #     use_equal_batcher=False,
    #     seed=420,
    #     label_type=label,
    #     final_label_factor=5/1000
    # )

    # loading dataset
    dataset, df_index = load_pure_ferris_wheel_dataset_single_node(
        node_type=node_type,
        num_elem=size_elem,
        num_to_generate=config.ds_size,
        dataset_path=dataset_path,
        final_label_factor=config.final_label_factor,
        normalize=True if config.dataset == -1 else 0
    )
    # splitting off test dataset
    generator = torch.Generator().manual_seed(config.seed)
    train_ds, val_ds, _ = random_split(
        dataset,
        [1 - test_perc - val_perc, val_perc, test_perc],
        generator=generator
    )

    # loaders
    train_loader = choice_trainloader(config, df_index, train_ds)
    val_loader = DataLoader(val_ds, batch_size=4 * config.batch_size, shuffle=False)

    # define device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # set seed for reproducability
    seeding_all(config.seed)

    # define model
    model = LayerManagementFramework(
        in_dim=config.in_dim,
        tree=TreeNode(node_type, [TreeNode("E", size_elem)]),
        hidden_dim=config.hidden_dim,
        out_dim=config.out_dim,
        k=config.k
    ).to(device)

    # learning tools
    criterion = choice_loss(config)
    optimizer = choice_optim(config, model)

    # init score list
    score_list = []

    # train for specified number of epochs
    for epoch in range(config.num_max_epochs):
        _, _, _ = train_epoch(
            model,
            train_loader,
            optimizer,
            device,
            criterion
        )

        _, val_true_y, val_pred_y = eval_epoch(
            model,
            val_loader,
            device,
            criterion
        )

        # calc r2 score and append
        score = r2_score(y_true=val_true_y, y_pred=val_pred_y)
        score_list += [score]
        trial.report(score, epoch)

    # calculate objective
    # display(score_list)
    # idea: last x r2 scores (why not last one? for stability purposes)
    obj = np.array(score_list)[-stability_count:]
    return np.mean(obj)

## Parameter search

### S

In [5]:
node_type = "S"
size_elem = 10
n_trials = 15
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 18:32:26,923] A new study created in memory with name: Study node type S, normalized: True


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2023-12-20 18:33:28,931] Trial 0 finished with value: -10.281744002578927 and parameters: {'hidden_dim': 160, 'k': 1, 'learning_rate': 4.959396653281266e-05, 'batcher': True}. Best is trial 0 with value: -10.281744002578927.
[I 2023-12-20 18:34:28,181] Trial 1 finished with value: 0.7466127936151782 and parameters: {'hidden_dim': 16, 'k': 2, 'learning_rate': 0.0006717627458406179, 'batcher': False}. Best is trial 1 with value: 0.7466127936151782.
[I 2023-12-20 18:35:28,921] Trial 2 finished with value: 0.8889506321931296 and parameters: {'hidden_dim': 48, 'k': 3, 'learning_rate': 0.00271345019195655, 'batcher': True}. Best is trial 2 with value: 0.8889506321931296.
[I 2023-12-20 18:36:32,135] Trial 3 finished with value: -105.376945730172 and parameters: {'hidden_dim': 128, 'k': 3, 'learning_rate': 2.3809863039752865e-05, 'batcher': True}. Best is trial 2 with value: 0.8889506321931296.
[I 2023-12-20 18:37:33,661] Trial 4 finished with value: -148.02068563405408 and parameters: {'hi

In [6]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_normalized-lesser-search.csv")

### Q

In [7]:
node_type = "Q"
size_elem = 10
n_trials = 15
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 19:11:42,892] A new study created in memory with name: Study node type Q, normalized: True


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2023-12-20 19:12:50,140] Trial 0 finished with value: 0.8957352031134302 and parameters: {'hidden_dim': 192, 'k': 2, 'learning_rate': 0.0012684148084059828, 'batcher': False}. Best is trial 0 with value: 0.8957352031134302.
[I 2023-12-20 19:13:57,095] Trial 1 finished with value: 0.8318871599730928 and parameters: {'hidden_dim': 224, 'k': 3, 'learning_rate': 5.420419499334039e-05, 'batcher': False}. Best is trial 0 with value: 0.8957352031134302.
[I 2023-12-20 19:15:02,091] Trial 2 finished with value: 0.8718999958219735 and parameters: {'hidden_dim': 112, 'k': 3, 'learning_rate': 0.0005609897351977381, 'batcher': True}. Best is trial 0 with value: 0.8957352031134302.
[I 2023-12-20 19:16:04,967] Trial 3 finished with value: 0.7104050081561926 and parameters: {'hidden_dim': 80, 'k': 1, 'learning_rate': 0.00035235944013280754, 'batcher': True}. Best is trial 0 with value: 0.8957352031134302.
[I 2023-12-20 19:17:10,829] Trial 4 finished with value: -143.36978767970646 and parameters: {

In [8]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_normalized-lesser-search.csv")

### C

In [9]:
node_type = "C"
size_elem = 10
n_trials = 15
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 19:28:21,404] A new study created in memory with name: Study node type C, normalized: True


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2023-12-20 19:29:32,723] Trial 0 finished with value: 0.8943885569228552 and parameters: {'hidden_dim': 176, 'k': 1, 'learning_rate': 0.0004842769229928358, 'batcher': False}. Best is trial 0 with value: 0.8943885569228552.
[I 2023-12-20 19:30:44,284] Trial 1 finished with value: -169.28916471684647 and parameters: {'hidden_dim': 144, 'k': 2, 'learning_rate': 1.0689100665773137e-06, 'batcher': True}. Best is trial 0 with value: 0.8943885569228552.
[I 2023-12-20 19:31:55,705] Trial 2 finished with value: 0.8613703972558632 and parameters: {'hidden_dim': 16, 'k': 3, 'learning_rate': 0.003975301910607491, 'batcher': True}. Best is trial 0 with value: 0.8943885569228552.
[I 2023-12-20 19:33:12,259] Trial 3 finished with value: 0.9765075975984564 and parameters: {'hidden_dim': 208, 'k': 1, 'learning_rate': 0.004582193274559827, 'batcher': True}. Best is trial 3 with value: 0.9765075975984564.
[I 2023-12-20 19:34:19,963] Trial 4 finished with value: -156.77195957005478 and parameters: {'h

In [10]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_normalized-lesser-search.csv")

### P

In [11]:
node_type = "P"
size_elem = 10
n_trials = 15
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-20 19:45:12,084] A new study created in memory with name: Study node type P, normalized: True


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2023-12-20 19:46:13,493] Trial 0 finished with value: 0.8766683852333849 and parameters: {'hidden_dim': 144, 'k': 3, 'learning_rate': 0.0021422901549198554, 'batcher': False}. Best is trial 0 with value: 0.8766683852333849.
[I 2023-12-20 19:47:12,644] Trial 1 finished with value: -160.382928660029 and parameters: {'hidden_dim': 48, 'k': 1, 'learning_rate': 1.660585125055431e-05, 'batcher': True}. Best is trial 0 with value: 0.8766683852333849.
[I 2023-12-20 19:48:14,195] Trial 2 finished with value: -160.91579826701224 and parameters: {'hidden_dim': 224, 'k': 2, 'learning_rate': 1.005802966990337e-06, 'batcher': True}. Best is trial 0 with value: 0.8766683852333849.
[I 2023-12-20 19:49:21,833] Trial 3 finished with value: 0.9718325757736146 and parameters: {'hidden_dim': 112, 'k': 1, 'learning_rate': 0.007898858792471212, 'batcher': True}. Best is trial 3 with value: 0.9718325757736146.
[I 2023-12-20 19:50:36,546] Trial 4 finished with value: -160.87552039661063 and parameters: {'hi

In [12]:
study.trials_dataframe().to_csv(f"csvs/single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}_normalized-lesser-search.csv")